In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('..')

import autorobot as ar
from autorobot.constants import (
    RProjType,
)

In [4]:
rb = ar.initialize()

In [5]:
rb.new(RProjType.SHELL)

In [6]:
rb.nodes.create(0, 0, 1)
rb.nodes.create(0, 1, 0)
rb.nodes.create(1, 0, 0)

In [7]:
node = rb.nodes.get('2')
print(node.as_array())

[0. 1. 0.]


In [8]:
for n in rb.nodes.select('1to3'):
    print(n.as_array())

[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]


In [9]:
ar.distance(1, '3')

1.4142135623730951

In [10]:
with rb.nodes as nodes:
    print(nodes.get(1))

Node 1: [0. 0. 1.]


In [11]:
rb.bars.create(1, 2)

In [12]:
n = rb.nodes.create(3., 0., 0.)
n = rb.nodes.create(2., 0., 0.)
assert(n.X == 2.)
print(int(n))

5


In [13]:
rb.nodes.create(0., 1., 3., num=1)

In [15]:
print(rb.nodes.get(1))

Node 1: [0. 1. 3.]
